In [1]:
import pickle
# Load split data
with open('split_data.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator instance for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    rotation_range=20,  # Rotate images randomly by up to 20 degrees
    width_shift_range=0.2,  # Shift images horizontally by up to 20% of the width
    height_shift_range=0.2,  # Shift images vertically by up to 20% of the height
    shear_range=0.2,  # Apply shear transformation with a maximum shear intensity of 20%
    zoom_range=0.2,  # Zoom in on images by up to 20%
    horizontal_flip=True,  # Flip images horizontally
    vertical_flip=True,  # Flip images vertically
    fill_mode='nearest'  # Fill in missing pixels with the nearest value
)

# Define batch size
batch_size = 32

# Prepare the training data generator
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)

In [1]:
from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, ReLU, BatchNormalization, add,Softmax, AveragePooling2D, Dense, Input, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
import os
import shutil
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

In [3]:
def expansion_block(x,t,filters,block_id):
    prefix = 'block_{}_'.format(block_id)
    total_filters = t*filters
    x = Conv2D(total_filters,1,padding='same',use_bias=False, name = prefix +'expand')(x)
    x = BatchNormalization(name=prefix +'expand_bn')(x)
    x = ReLU(6,name = prefix +'expand_relu')(x)
    return x

def depthwise_block(x,stride,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = DepthwiseConv2D(3,strides=(stride,stride),padding ='same', use_bias = False, name = prefix + 'depthwise_conv')(x)
    x = BatchNormalization(name=prefix +'dw_bn')(x)
    x = ReLU(6,name=prefix +'dw_relu')(x)
    return x

def projection_block(x,out_channels,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = Conv2D(filters = out_channels,kernel_size = 1,padding='same',use_bias=False,name= prefix + 'compress')(x)
    x = BatchNormalization(name=prefix +'compress_bn')(x)
    return x

In [4]:
def Bottleneck(x,t,filters, out_channels,stride,block_id):
    y = expansion_block(x,t,filters,block_id)
    y = depthwise_block(y,stride,block_id)
    y = projection_block(y, out_channels,block_id)
    if y.shape[-1]==x.shape[-1]:
        y = add([x,y])
    return y

In [5]:
def MobileNetV2(input_image = (224,224,3), n_classes=1000):
    input = Input(input_image)

    x = Conv2D(32,kernel_size=3,strides=(2,2),padding = 'same', use_bias=False)(input)
    x = BatchNormalization(name='conv1_bn')(x)
    x = ReLU(6, name = 'conv1_relu')(x)

    # 17 Bottlenecks

    x = depthwise_block(x,stride=1,block_id=1)
    x = projection_block(x, out_channels=16,block_id=1)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 24, stride = 2,block_id = 2)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 24, stride = 1,block_id = 3)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 2,block_id = 4)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 5)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 6)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 2,block_id = 7)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 8)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 9)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 10)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 11)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 12)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 13)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 2,block_id = 14)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 15)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 16)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 320, stride = 1,block_id = 17)


    #1*1 conv
    x = Conv2D(filters = 1280,kernel_size = 1,padding='same',use_bias=False, name = 'last_conv')(x)
    x = BatchNormalization(name='last_bn')(x)
    x = ReLU(6,name='last_relu')(x)

    #AvgPool 7*7
    x = GlobalAveragePooling2D(name='global_average_pool')(x)

    output = Dense(n_classes,activation='softmax')(x)

    model = Model(input, output)

    return model

In [6]:
n_classes = 6
input_shape = (224,224,3)

model = MobileNetV2(input_shape,n_classes)
model.summary()



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)         128       ['conv2d[0][0]']              
 on)                                                                                              
                                                                                                  
 conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['conv1_bn[0][0]']      

In [7]:

# Set the paths to your dataset folders
qr_code_path1 = r'C:\Users\ilfas\OneDrive\Documents\college\design project\implementation\qr_codes2'
target_path = r'C:\Users\ilfas\OneDrive\Documents\college\design project\implementation\output_qr_codes'

# Function to load images without preprocessing
def load_images(directory):
    images = []
    for img in os.listdir(directory):
        img_path = os.path.join(directory, img)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Adjust the size according to your needs
        img = img[..., ::-1]  # BGR to RGB (since MobileNetV2 expects RGB)
        img = img / 255.0  # Normalize pixel values to the range [0, 1]
        images.append(img)
    return np.array(images)

# Load QR code images and target images
qr_code_images = load_images(qr_code_path1)
target_images = load_images(target_path)

# Assuming you have labels for your QR code images (modify accordingly)
labels = np.array([1] * 1000 + [2] * 1000 + [3] * 1000 + [4] * 1000 + [5] * 1000 + [6] * 1000)

# Subtract 1 to bring labels into the range [0, 5]
labels = labels - 1


MemoryError: Unable to allocate 6.73 GiB for an array with shape (6000, 224, 224, 3) and data type float64

In [ ]:
# Split the dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(qr_code_images, labels, test_size=0.3, random_state=0)

In [14]:
import pickle

# Save split data
with open('split_data1.pkl', 'wb') as f:
    pickle.dump((X_train, y_train, X_test, y_test), f)

In [ ]:
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [ ]:

# Define optimizer, loss function, and metrics
optimizer = Adagrad(learning_rate=0.01)  # You can adjust the learning rate as needed
loss_function = SparseCategoricalCrossentropy()
metrics = [SparseCategoricalAccuracy()]

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

# Print model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)         128       ['conv2d[0][0]']              
 on)                                                                                              
                                                                                                  
 conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['conv1_bn[0][0]']        

In [11]:
# Directory path containing QR code images
qr_code_path = 'qr_codes2'

# Function to load images
def load_images(directory):
    images = []
    for img_name in os.listdir(directory):
        img_path = os.path.join(directory, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize images to match MobileNetV2 input shape
        images.append(img)
    return np.array(images)

# Load QR code images
qr_code_images = load_images(qr_code_path)
labels = np.array([1] * 1000 + [2] * 1000 + [3] * 1000 + [4] * 1000 + [5] * 1000 + [6] * 1000)

# Subtract 1 to bring labels into the range [0, 5]
labels = labels - 1

# Train the ShuffleNet model on QR code images
model.fit(qr_code_images, labels, epochs=16, batch_size=32, validation_split=0.2)


Epoch 1/16

150/150 [==============================] - 241s 2s/step - loss: 1.1624 - sparse_categorical_accuracy: 0.5173 - val_loss: 3.1487 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/16
150/150 [==============================] - 219s 1s/step - loss: 0.8788 - sparse_categorical_accuracy: 0.6308 - val_loss: 4.4309 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/16
150/150 [==============================] - 262s 2s/step - loss: 0.7325 - sparse_categorical_accuracy: 0.6950 - val_loss: 5.1440 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/16
150/150 [==============================] - 265s 2s/step - loss: 0.6564 - sparse_categorical_accuracy: 0.7256 - val_loss: 7.0544 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/16
150/150 [==============================] - 323s 2s/step - loss: 0.5745 - sparse_categorical_accuracy: 0.7615 - val_loss: 5.7827 - val_sparse_categorical_accuracy: 0.1083
Epoch 6/16
150/150 [==============================] - 292s 2s/step - loss: 0.4

In [ ]:
from tensorflow.keras import callbacks, optimizers
import time

# Define a custom callback to measure training time
class TimingCallback(callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()

    def on_train_end(self, logs=None):
        end_time = time.time()
        training_time = end_time - self.start_time
        print(f"Training time: {training_time} seconds")

# Define a custom callback to measure training time
timing_callback = TimingCallback()

# Assuming X_train, y_train, X_val, y_val are your training and validation data
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=16, batch_size=32, callbacks=[timing_callback])


Epoch 1/16

132/132 [==============================] - 263s 2s/step - loss: 1.4059 - sparse_categorical_accuracy: 0.4157 - val_loss: 1.8017 - val_sparse_categorical_accuracy: 0.1661
Epoch 2/16
132/132 [==============================] - 248s 2s/step - loss: 1.0867 - sparse_categorical_accuracy: 0.5500 - val_loss: 1.8650 - val_sparse_categorical_accuracy: 0.1583
Epoch 3/16
132/132 [==============================] - 201s 2s/step - loss: 0.9075 - sparse_categorical_accuracy: 0.6143 - val_loss: 1.8164 - val_sparse_categorical_accuracy: 0.1583
Epoch 4/16
132/132 [==============================] - 174s 1s/step - loss: 0.8120 - sparse_categorical_accuracy: 0.6645 - val_loss: 1.9407 - val_sparse_categorical_accuracy: 0.1583
Epoch 5/16
132/132 [==============================] - 126s 957ms/step - loss: 0.7164 - sparse_categorical_accuracy: 0.7050 - val_loss: 2.4355 - val_sparse_categorical_accuracy: 0.2517
Epoch 6/16
132/132 [==============================] - 172s 1s/step - loss: 0.6036 - sparse_

In [ ]:
# Save the model
model.save('C:/Users/ilfas/OneDrive/Documents/college/design project/implementation/MobileNet.h5')

In [1]:
from sklearn.metrics import classification_report

# Make predictions on the training data
train_predictions = model.predict(X_train)
train_pred_labels = np.argmax(train_predictions, axis=1)

# Generate classification report
train_classification_report = classification_report(y_test, train_pred_labels)

print("Training Classification Report:")
print(train_classification_report)


NameError: name 'model' is not defined

In [ ]:
report_dict = train_classification_report(y_true, y_pred, target_names=target_names, output_dict=True)

# Get specificity from the report
specificity = report_dict['class_name']['specificity']

In [14]:
model = Model(inputs=model.input, outputs=model.layers[-2].output)  # Get the second-to-last layer (softmax layer)
#features = model.predict(qr_code_images)
# Get the features exclusively from the second-to-last layer
features = model.predict(qr_code_images)[:2000]


# Reshape the features to have only two dimensions
num_samples, height, width, num_channels = features.shape
features_reshaped = features.reshape(num_samples, height * width * num_channels)

# Step 2: Training and Testing Split
X_train, X_test, y_train, y_test = train_test_split(features_reshaped, labels, test_size=0.3, random_state=42)


188/188 [==============================] - 37s 185ms/step


In [13]:
model = Model(inputs=model.input, outputs=model.layers[-2].output)  # Get the second-to-last layer (softmax layer)
#features = model.predict(qr_code_images)
# Get the features exclusively from the second-to-last layer
features = model.predict(qr_code_images)


# Reshape the features to have only two dimensions
num_samples, height, width, num_channels = features.shape
features_reshaped = features.reshape(num_samples, height * width * num_channels)



188/188 [==============================] - 24s 122ms/step


In [7]:
# Step 2: Training and Testing Split
X_train, X_test, y_train, y_test = train_test_split(features_reshaped, labels, test_size=0.3, random_state=42)

NameError: name 'labels' is not defined

In [18]:
import pickle

# Save features_reshaped to a file using pickle
with open('features_reshaped1.pkl', 'wb') as f:
    pickle.dump(features_reshaped, f)


In [16]:
import pickle

# Save features to a file using pickle
with open('features.pkl', 'wb') as f:
    pickle.dump(features, f)


In [3]:
import pickle
with open('features_reshaped1.pkl', 'rb') as f:
    features_reshaped = pickle.load(f)

In [15]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [16]:
#new
# Train SVM classifier using the extracted features
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.8638888888888889

In [14]:
#old
# Train SVM classifier using the extracted features
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.85

In [17]:
#new
# Train SVM classifier using the extracted features
svm_classifier = SVC(kernel='poly', degree=2, C=1.0, gamma='auto')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.8527777777777777

In [15]:
#old
# Train SVM classifier using the extracted features
svm_classifier = SVC(kernel='poly', degree=2, C=1.0, gamma='auto')
svm_classifier.fit(X_train, y_train)


# Evaluate classifiers on the testing set
svm_accuracy = svm_classifier.score(X_test, y_test)
svm_accuracy

0.855

In [16]:
# Train kNN classifier using the extracted features
knn_classifier = KNeighborsClassifier(metric='euclidean')
knn_classifier.fit(X_train, y_train)

knn_accuracy = knn_classifier.score(X_test, y_test)
knn_accuracy

0.8422222222222222

In [17]:
# Train kNN classifier using the extracted features
knn_classifier = KNeighborsClassifier(metric='manhattan')
knn_classifier.fit(X_train, y_train)

knn_accuracy = knn_classifier.score(X_test, y_test)
knn_accuracy

0.8511111111111112

In [18]:
# Train kNN classifier using the extracted features
knn_classifier = KNeighborsClassifier(metric='minkowski')
knn_classifier.fit(X_train, y_train)

knn_accuracy = knn_classifier.score(X_test, y_test)
knn_accuracy

0.8422222222222222